In [ ]:
!pip install datasets pandas numpy matplotlib google-colab
!pip install --upgrade --upgrade-strategy eager "regex" "charset-normalizer<4" "idna" "urllib3<3" "certifi" "requests" "anyio<5" "distro<2" "sniffio" "h11<0.15" "httpcore==1.*" "annotated-types" "typing-extensions<5" "pydantic-core==2.27.1" "pydantic<3" "jiter<1" "tqdm" "colorama" "openai" "tiktoken" "httpx<0.28"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and 

In [ ]:
from datasets import load_dataset
import openai
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
from openai import OpenAIError
import argparse
from datetime import datetime
from tqdm import tqdm
import json
import os
import random

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# do not share
api_key = ""
client = OpenAI(
    api_key=api_key,
)
def run_gpt4(prompt):
    try:
        response = client.chat.completions.create(model="gpt-4o", messages=prompt, temperature = 0, max_tokens=4096)
    except Exception as e:
        print(e)
        return run_gpt4(prompt)
    return response

In [ ]:
def prepare_prompt_Final(question, answer, unit, section, lesson, unit_narrative, section_narrative, lesson_narrative, position = 1):
   messages = [{"role": "system", "content": f'''You are a tutor designed to help students solve math problems. Your goal is to generate tutoring for the given problem. If necessary, you may also generate subtutoring for difficult tutoring. The tutoring and subtutoring must obey the following rules:
1. At the beginning, include a general outline that briefly restates or reframes the problem to clarify the goal or general idea to help students understand the problem. Keep the general outline simple and avoid wordiness. The general outline must follow this format: The first part should tell students what the goal of the problem is. The second part should tell students the information in the problem that they should use to find the answer presented in a bulleted list of information; this information must be directly from the problem itself.
2. Tutoring should engage students in the problem-solving process without giving away the answer until students have thought about it and tried. Tutoring should ask the student to think on their own, and subtutoring should provide more direct support.
3. The main tutoring must be a Tutoring Question. Subtutoring must be a Hint followed by an Explanation:
   - Hints: Provide a small amount of guidance about the concept needed to solve the current question. **Hints should be statements** and **must not ask a question or require an answer**.
   - Explanations: Fully explain how to solve the tutoring question. **Explanations must be statements**, should give a step-by-step procedure, and **must not ask a question or require an answer**.
The last tutoring question should use a fill-in-the-blank format and must not be a 'How' question, as defined in the 'How' Question Rule section later in this prompt. The answer to this last tutoring question must match the answer to the original problem.
5. Each tutoring should include as much subtutoring as needed. If  the tutoring question is complex and subtutoring is needed, Subtutoring x.1 should be a Hint, and x.2 should be an Explanation to give students a chance to think and work through on their own and guide the solution effectively. If needed, Subtutoring x.3 should be another Hint and x.4 should be an Explanation. If the tutoring question isn’t complex, a hint may not be needed, and x.3 and x.4 may not be needed.
6. Explanations MUST include the correct answer to the Tutoring Question within the explanation. The explanation for the last Tutoring Question should include the answer to the problem within the explanation.
7. Each question must follow these rules:
    Fill-in-the-blank rule:
    ONLY use a fill-in-the-blank format if the correct answer is a single number or a single standalone word.
    DO NOT provide multiple-choice options in this case.
    DO NOT allow full phrases, steps, or explanations in a fill-in-the-blank question.
    Leave the options field empty like this: options: [].
    STRICT ENFORCEMENT: If the correct answer is longer than one word or one number, it MUST be multiple choice.
    Examples:
    ✅ Fill-in-the-blank: “What is 8 × 7?” → Answer: 56 → options: []
    ✅ Fill-in-the-blank: “What is the term for a 10-sided polygon?” → Answer: Decagon → options: []
    ❌ Incorrect fill-in-the-blank: “What is the first step in solving for x?” → Answer: Substitute 5 for x in the equation (This must be multiple-choice!)
    Multiple-choice rule:
    If the answer is a phrase, step, or explanation longer than a single word or number, it MUST be a multiple-choice question.
    Provide 2 or 3 distinctly different answer choices based on complexity.
    Exactly one option must be correct.
    Example:
    ✅ Multiple-choice: “What is the first step to find the number of packets needed for 30,000 bytes using the equation b=1,500p?”
    A) Substitute 30,000 for b in the equation ✅ (Correct)
    B) Divide b by 30,000
    C) Multiply p by 1,500
    Example:
    ✅ Multiple-choice: “How can you calculate the total number of grapes eaten if you know the number of grapes eaten per minute and the total number of minutes?”
    A) Multiply the number of grapes per minute by the number of minutes ✅ (Correct)
    B) Add the number of grapes per minute to the total number of minutes
    C) Divide the number of minutes by the number of grapes per minute
    Equation & Expression rule:
    Always use multiple-choice for equations and expressions.
    The left side of the equation must always match what the question is asking about.
    All answer choices must have the exact same left-hand side to ensure consistency.
    Example: If the question asks for the equation of a line in slope-intercept form (y = mx + b), then all options must follow this structure:
    ✅ y = 2x + 3
    ✅ y = -4x + 1
    ✅ y = 5x - 7
    ❌ x = (y - 3)/2 (wrong format)
    ❌ y - 3 = 2x (equivalent to y = 2x + 3)
    Mathematical Uniqueness Rule:
    The multiple-choice options must not be mathematically equivalent in any form.
    Do not include options that can be simplified, rearranged, or transformed into each other.
    ✅ Correct: x = 2a + 3, x = 4a - 1, x = 5a/2
    ❌ Incorrect: x = 2(a + 1), x = 2a + 2 (since these are equivalent)
    Avoid fractions, ratios, or numerical values that simplify to the same number.
    ✅ Correct: x = 3/5, x = 4/7, x = 2/9
    ❌ Incorrect: x = 1/4, x = 2/8, x = 25% (all are the same value)
    "How" Question Rule:
    If a "How" question results in a numerical answer, it must be fill-in-the-blank.
    If a "How" question requires a step, method, or explanation, it must be multiple-choice.
    DO NOT assume all "How" questions require a multiple-choice format—first, check if the answer is numerical.
    If the model is about to generate a multiple-choice question for a 'How' question with a numerical answer, STOP and correct it to fill-in-the-blank.
    If the model is about to generate a fill-in-the-blank for a 'How' question that requires an explanation or step, STOP and correct it to multiple-choice.
    Examples (Fixing All "How" Question Issues):
    ✅ Fill-in-the-blank (numerical "How" question):
    "How long does it take to travel 300 miles at a speed of 60 mph?"
    Answer: 5
    options: []
    ✅ Fill-in-the-blank (other numerical 'How' questions beyond 'How many' and 'How much'):
    "How far does a car travel in 3 hours at 50 mph?"
    Answer: 150
    options: []
    "How high is the triangle if the base is 10 and the area is 50?"
    Answer: 10
    options: []
    ✅ Multiple-choice (procedural 'How' question requiring a method/step):
    "How can you calculate the total number of grapes eaten if you know the number of grapes eaten per minute and the total number of minutes?"
    A) Multiply the number of grapes per minute by the number of minutes ✅
    B) Add the number of grapes per minute to the number of minutes
    C) Divide the number of grapes per minute by the number of minutes
    ✅ Multiple-choice (procedural 'How' question requiring an explanation):
    "How can you convert the time Clare cycles from minutes to hours?"
    A) Divide the number of minutes by 60 ✅
    B) Multiply the number of minutes by 2
    C) Add 10 to the number of minutes
    Verification, Correction & Enforcement Rule:
    Double-check that all answer choices are unique in structure and meaning.
    No two options should express the same mathematical relationship in a different form.
    If the model is about to generate a multi-word answer as a fill-in-the-blank, STOP and correct it to multiple-choice.
    If the model is about to generate mathematically equivalent multiple-choice options, STOP and regenerate unique options.
    If the model is about to generate an equation where the left side does not match what the question asks about, STOP and correct it.
    All errors must be fixed before finalizing the question!
    Failure to follow these rules must result in immediate correction before generating the final question.
8. Hints and Explanations must be statements, not questions, and should guide or explain the solution process without requiring a response.
9. Aim to minimize the length of multiple-choice options to avoid overwhelming students.
10. Tutoring should explain the rationale for the major steps rather than only providing the procedure. It should help reduce cognitive load by reminding students of relevant information from the problem or from prior tutoring.
11. The number of tutoring questions should change based on the complexity of the problem. For example, simpler problems might need 1-2 tutoring questions (with sub-tutoring as needed) and more complex problems might need 3-5 tutoring questions (with sub-tutoring as needed).
12. Each tutoring question should account for the prior tutoring questions and the general outline and use parallel strategies for solving the problem.
13. 🚫 Strict Prohibition on Operation-Based Questions:
   Tutoring questions must NEVER ask students what mathematical operation to use (e.g., addition, subtraction, multiplication, or division).
   DO NOT generate questions such as:
       ❌  "What operation should you use to find the distance traveled by...?"
       ❌"What operation should you perform to find ...?"
       ❌"What operation should you use to find the total weight of..."
       ❌ "What operation helps you determine the difference between..."
       ❌ "Which mathematical operation should be applied to solve this problem?"
   Questions like these often have multiple-choice options (e.g., addition, subtraction, multiplication, division), and they must NOT be included in the output.
   If the model is about to generate a question like this, STOP and rephrase it so that it guides students toward understanding concepts without directly asking about operations.
14. Use mathematical expressions when possible to reduce wordiness.
15. This is part of unit {unit} section {section} lesson {lesson}. To aid, please reference the unit narrative: {unit_narrative} the section narrative: {section_narrative} and lesson narrative {lesson_narrative}.
Please follow the procedures and use the conventions in the unit and lesson. Please ensure that you use topics in this unit and lesson and do not use more advanced topics which are out of scope of this lesson.
You must output the tutoring in a JSON format as follows:
{{
 "problem": "Original problem text",
 "answer": "Original problem answer",
 "general outline": "Briefly restate or reframe the problem to clarify the goal or general idea by explaining the goal of the problem, then providing a bulleted list of relevant information directly from the problem.",
 "tutoring": [
 {{
     "number": "1",
     "body": "Tutoring Question 1 text",
     "type": "Question",
     "options": ["option 1", "option 2", "optional option 3"], # if the answer is number or single word then leave options blank
     "answer": "Answer to Tutoring Question 1",
     "subtutoring": [
       {{
         "number": "1.1",
         "body": "Hint for subtutoring 1.1",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "1.2",
         "body": "Explanation for subtutoring 1.2",
         "type": "Explanation",
         "options": [],
         "answer": ""
       }},
       {{
         "number": "1.3",
         "body": "Hint for subtutoring 1.3",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "1.4",
         "body": "Explanation for subtutoring 1.4",
         "type": "Explanation",# if applicable
         "options": [],
         "answer": ""
       }},
 ...,
      {{
         "number": "1.n",
         "body": "Hint for subtutoring 1.n",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "1.(n+1)",
         "body": "Explanation for subtutoring 1.(n+1)",
         "type": "Explanation",# if applicable
         "options": [],
         "answer": ""
       }}
     ]
   }},
   {{
     "number": "2",
     "body": "Tutoring Question 2 text",
     "type": "Question",
     "options": ["option 1", "option 2", "optional option 3"], # if applicable
     "answer": "Answer to Tutoring Question 2",
     "subtutoring": [
       {{
         "number": "2.1",
         "body": "Hint for subtutoring 2.1",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "2.2",
         "body": "Explanation for subtutoring 2.2",
         "type": "Explanation",
         "options": [],
         "answer": ""
       }},
       {{
         "number": "2.3",
         "body": "Hint for subtutoring 2.3",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "2.4",
         "body": "Explanation for subtutoring 2.4",
         "type": "Explanation",# if applicable
         "options": [],
         "answer": ""
       }},
 ...,
      {{
         "number": 2.n",
         "body": "Hint for subtutoring 2.n",
         "type": "Hint",# if applicable
         "options": [],
         "answer": ""
       }},
       {{
         "number": "2.(n+1)",
         "body": "Explanation for subtutoring 2.(n+1)",
         "type": "Explanation",# if applicable
         "options": [],
         "answer": ""
       }}
     ]
   }},
   ...,
   {{
         "number": "n",
         "body": "Tutoring Question n text",
         "type": "Question",
         "options": ["option 1", "option 2", "optional option 3"], # if applicable
         "answer": "Answer to Tutoring Question n",
         "subtutoring": [
          {{
             "number": "n.1",
             "body": "Hint for subtutoring n.1",
             "type": "Hint",# if applicable
             "options": [],
             "answer": ""
           }},
           {{
             "number": "n.2",
             "body": "Explanation for subtutoring n.2",
             "type": "Explanation",
             "options": [],
             "answer": ""
           }},
           {{
             "number": "n.3",
             "body": "Hint for subtutoring n.3",
             "type": "Hint",# if applicable
             "options": [],
             "answer": ""
       }},
          {{
             "number": "n.4",
             "body": "Explanation for subtutoring n.4",
             "type": "Explanation",# if applicable
             "options": [],
             "answer": ""
       }},
 ...,
         {{
            "number": "n.n",
            "body": "Hint for subtutoring n.n",
            "type": "Hint",# if applicable
            "options": [],
            "answer": ""
         }},
         {{
         "number": "n.n",
         "body": "Explanation for subtutoring n.(n+1)",
         "type": "Explanation",# if applicable
         "options": [],
         "answer": ""
         }}
         ]
       }}
     ]
   }}
 ]
}}
'''},
           {"role": "user", "content": f'''Here is the question given to students: {question}. Here is the answer: {answer}.
            Please provide tutoring and subtutoring to help a student solve that problem.'''}
       ]
   return messages


In [ ]:
unit = input('Enter the unit name i.e. Unit 1, Unit 2, etc...: ')

Enter the unit name i.e. Unit 1, Unit 2, etc...: Unit 2


In [ ]:
shared_drive_path = r'/content/gdrive/Shareddrives/CAIT -  Transformative IES Grant/TAF-WPI - Inner Loop CAIT/Scaffolding Generation/'
df = pd.read_csv(shared_drive_path + 'grade7clean.csv')
df = df[df['problem_path'].str.contains(unit)]
df = df.reset_index(drop=True)
narrative = pd.read_csv(shared_drive_path + 'Narrative.csv')
narrative.columns = [col.rstrip() for col in narrative.columns]


In [ ]:
print(shared_drive_path)

/content/gdrive/Shareddrives/CAIT -  Transformative IES Grant/TAF-WPI - Inner Loop CAIT/Scaffolding Generation/


In [ ]:
max_index = input(f'How many would you like to generate? (out of {len(df)}) ')

How many would you like to generate? (out of 59) 10


In [ ]:
df['unit'] = df['problem_path'].str.extract(r'(Unit \d+)')
df['lesson'] = df['problem_path'].str.extract(r'(Lesson \d+)')

In [ ]:
for index, row in df.iterrows():
    if index >= int(max_index):
        break
    if index % 10 == 0:
        print(index)
    question = row['cleaned body']
    answer = row['answer_value']
    unit = row['unit']
    lesson = row['lesson']

    # Extract unit narrative
    unit_narrative_column = narrative.columns[narrative.columns.str.contains(f"^{unit}$", regex=True)]
    unit_narrative = narrative[unit_narrative_column[0]].iloc[0] if not unit_narrative_column.empty else None

    # Find the section
    section_row = narrative.loc[
        narrative.apply(
            lambda col: col.astype(str).str.contains(f"^{unit} Section .* {lesson}$", regex=True, na=False)
        ).any(axis=1)
    ]
    section = section_row.columns[0] if not section_row.empty else None

    # Extract section name and narratives
    if section:
        section_name = section.split(' ')[1]  # Extract Section A, Section B, etc.
        section_narrative = narrative[section].iloc[0]
    else:
        section_name = None
        section_narrative = None

    # Extract lesson narrative
    lesson_column = narrative.columns[narrative.columns.str.contains(f"^{unit} .* {lesson}$", regex=True)]
    lesson_narrative = narrative[lesson_column[0]].iloc[0] if not lesson_column.empty else None

    # Get position
    position = 1 # row['position'] if pd.notnull(row['position']) else 1

    # Prepare and run prompt
    prompt = prepare_prompt_Final(question, answer, unit, lesson, section_name, unit_narrative, lesson_narrative, section_narrative, position)
    response = run_gpt4(prompt)
    resp = response.choices[0].message.content.strip()

    # Update dataframe
    df.at[index, 'Tutoring'] = resp
    df.at[index, 'TutoringPrompt'] = prompt[0]['content']


0


In [ ]:
for x in narrative.columns:
  print(x)

Unit 6
Unit 6 Section A Goals
Unit 6 Section A Narartive
Unit 6 Section A Lesson 1
Unit 6 Section A Lesson 2
Unit 6 Section A Lesson 3
Unit 6 Section A Lesson 4
Unit 6 Section A Lesson 5
Unit 6 Section A Lesson 6
Unit 6 Section B Goals
Unit 6 Section B Narrative
Unit 6 Section B Lesson 7
Unit 6 Section B Lesson 8
Unit 6 Section A Lesson 9
Unit 6 Section B Lesson 10
Unit 6 Section B Lesson 11
Unit 6 Section B Lesson 12
Unit 6 Section C Goals
Unit 6 Section C Narrative
Unit 6 Section C Lesson 13
Unit 6 Section C Lesson 14
Unit 6 Section C Lesson 15
Unit 6 Section C Lesson 16
Unit 6 Section C Lesson 17
Unit 6 Section D Goals
Unit 6 Section D Narrative
Unit 6 Section D Lesson 18
Unit 6 Section D Lesson 19
Unit 6 Section D Lesson 20
Unit 6 Section D Lesson 21
Unit 6 Section E Narrative
Unit 6 Section E Lesson 22
Unit 6 Glossary
Unit 5
Unit 5 Section A Goals
Unit 5 Section A Narrative
Unit 5 Section A Lesson 1
Unit 5 Section A Lesson 2
Unit 5 Section A Lesson 3
Unit 5 Section A Lesson 4
Unit

In [ ]:
def clean_text(text):
    if isinstance(text, float):  # If it's a float, return it as-is
        return text
    elif '```json' in text and '```' in text:
        # Find the first occurrence of '```json' and remove everything before it
        start_index = text.index('```json') + len('```json')
        text = text[start_index:].strip()

        # Find the last occurrence of '```' and remove everything after it
        end_index = text.rindex('```')
        text = text[:end_index].strip()

    return text


df['Tutoring'] = df['Tutoring'].apply(clean_text)

In [ ]:
def check_hints_explanations(json_data):
    issues = []  # Store issues found in the response

    def check_subtutoring(subtutoring):
        for sub in subtutoring:
            body = sub['body']
            if sub['type'] in ['Hint', 'Explanation'] and body.strip().endswith('?'):
                issues.append(f"Hint or Explanation ending with a question mark: {body}")
            # Recursively check if subtutoring exists
            if 'subtutoring' in sub:
                check_subtutoring(sub['subtutoring'])

    # Parse the tutoring and check each entry
    tutoring = json_data['tutoring']
    for item in tutoring:
        subtutoring = item['subtutoring']
        check_subtutoring(subtutoring)

    return issues

def check_responses(df):
    for index, response in df['Tutoring'].items():  # Get index for error reporting
        if isinstance(response, str):  # Check if response is a string before parsing
            try:
                json_data = json.loads(response)  # Parse the JSON in each response
                issues = check_hints_explanations(json_data)  # Check for issues in the JSON
                if issues:
                    print(f"Issues found in response at index {index}:")
                    for issue in issues:
                        print(issue)
            except json.JSONDecodeError:
                print(f"Invalid JSON at index {index}")
        elif pd.isnull(response):  # Check if response is NaN
            pass
        else:
            pass

check_responses(df)

In [ ]:
def prepare_prompt_summary(tutoring):
    parsed_data = json.loads(tutoring)

    # Count the number of main steps in the tutoring
    num_steps = len(parsed_data['tutoring'])
    messages = [{"role": "system", "content": f'''You are a tutor bot designed to summarize tutoring. You will be given tutoring and it is your job to summarize it into 2-5 word descriptions of each step. Each step of the scaffolding should correlate to one item in a list of steps.

    Output just a python list as follows:
    ["summary of step 1", "summary of step 2", ..., "summary of step n"]
    '''},
                {"role": "user", "content": f'''Please provide a summary of the steps for the following tutoring:
                {tutoring}
                Remember there must be exactly one step per main step, so there should be {num_steps} steps.'''}
            ]
    return messages


In [ ]:
for index, row in df.iterrows():
    if index >= int(max_index):
        break
    if (index % 10 == 0):
        print(index)
    tutoring = row['Tutoring']
    prompt = prepare_prompt_summary(tutoring)
    response = run_gpt4(prompt)
    resp = response.choices[0].message.content.strip()
    df.at[index, 'Steps'] = resp
    df.at[index, 'StepsPrompt'] = prompt[0]['content']

0


In [ ]:
def clean_text(text):
    if isinstance(text, float):  # If it's a float, return it as-is
        return text
    elif '```python' in text and '```' in text:
        # Find the first occurrence of '```json' and remove everything before it
        start_index = text.index('```python') + len('```python')
        text = text[start_index:].strip()

        # Find the last occurrence of '```' and remove everything after it
        end_index = text.rindex('```')
        text = text[:end_index].strip()

    return text


df['Steps'] = df['Steps'].apply(clean_text)

In [ ]:
def format_json_as_text(json_data):
    # Check if json_data is a string before processing, otherwise return empty string
    if isinstance(json_data, str):
        json_data = json_data.replace("\\n", '')  # Replace single quotes with double quotes
        json_data = json.loads(json_data)

        # Initialize result list to store lines
        result = []

        # Append problem, answer, and general outline
        result.append(f"Problem~{json_data['problem']}")
        result.append(f"Answer~{json_data['answer']}")
        result.append(f"0~General Outline~{json_data['general outline']}\n")

        # Iterate through tutoring
        for tutor in json_data['tutoring']:
            options = ", ".join(tutor['options']) if tutor['options'] else ""
            result.append(f"{tutor['number']}~{tutor['type']}~{tutor['body']}~{options}~{tutor['answer']}")

            # Iterate through subtutoring
            for sub in tutor['subtutoring']:
                result.append(f"{sub['number']}~{sub['type']}~{sub['body']}")

            result.append("")  # Blank line for separation

        return "\n".join(result)
    else:
        # Handle cases where json_data is not a string, such as returning an empty string
        return ""  # Or any other suitable default value

df['Table'] = df['Tutoring'].apply(format_json_as_text)

In [ ]:
df.to_csv(shared_drive_path + unit + 'NAME.csv')
print(shared_drive_path + unit + 'NAME.csv')

/content/gdrive/Shareddrives/CAIT -  Transformative IES Grant/TAF-WPI - Inner Loop CAIT/Scaffolding Generation/Unit 2tutoringtest 2025.04.10-Li test.csv


In [ ]:
df.columns

Index(['Unnamed: 0', 'index', 'problem_path', 'id', 'author_id', 'owner_id',
       'editor_id', 'visible_problem_type_id', 'name', 'body', 'properties',
       'created_at', 'updated_at', 'is_active', 'answer_value', 'problem_ceri',
       'cleaned body', 'unit', 'lesson', 'Tutoring', 'TutoringPrompt', 'Steps',
       'StepsPrompt', 'Table'],
      dtype='object')

In [ ]:
df[['problem_ceri', 'Tutoring', 'Steps']].to_csv(shared_drive_path + unit + 'NAME.csv')